In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import pi, sin, cos, dot

class ThinFilm_model:
    def __init__(self, ambient, thinfilm, substrate):
        self.ambient = ambient['a']
        self.thinfilm = thinfilm
        self.substrate = substrate['s']

    def Eq_TF_Matrix(self, wavelength):
        eq_matrix = identity_matrix(wavelength)
        for i in range(np.size(self.thinfilm)):
            eq_matrix = Matrix_dot(Matrix(self.thinfilm[-i-1], wavelength), eq_matrix)
        return eq_matrix

    def Eq_admittance(self, wl):
        eq_matrix = self.Eq_TF_Matrix(wl)
        Eq_y = BC(eq_matrix, self.substrate, wl)
        Y = Eq_y['C']/Eq_y['B']
        return Y.values.reshape(np.size(wl), 1)
        
    def R(self, wl):
        eq_Y = self.Eq_admittance(wl)
        r = (self.ambient-eq_Y)/(self.ambient+eq_Y)
        reflectance = np.reshape(r*r.conjugate(), np.size(eq_Y))
        return np.real(reflectance)

def BC(eq, ns, wl):
    bc = pd.DataFrame(np.reshape(dot(eq, np.array([[1], [ns]])), (np.size(wl), 2)),columns = ['B','C'])
    return bc

def chromatic_n(m, wl):
    n = np.ones(np.size(wl))
    return n

def Matrix(layer, wl):
    m = matrix(layer['m'], layer['d'], wl)
    return m

def matrix(m, t, wl):
    ita = chromatic_n(m, wl) 
    delta = 2*pi*ita*t/wl
    element = matrix_element(ita, delta)
    return np.reshape(element.values.reshape(1,-1), (np.size(wl), 2, 2))

def matrix_element(ita, delta):
    e = pd.DataFrame(
        {'e1':cos(delta), 'e2':1j/ita*sin(delta), 
         'e3':1j*ita*sin(delta), 'e4':cos(delta)})
    return e
    
def Matrix_dot(layer_up, layer_bot): 
    w, _, _ = np.shape(layer_up)
    eq = [dot(layer_up[i], layer_bot[i]) for i in range(w)]
    return eq    

def identity_matrix(wl):
    m = np.size(wl)
    i = pd.DataFrame({'e1':np.ones(m), 'e2':np.zeros(m), 'e3':np.zeros(m), 'e4':np.ones(m)})
    i_matrix = np.reshape(i.values.reshape(1,-1), (m, 2, 2))
    return i_matrix

model_1 = ThinFilm_model(
    {'a': 1}, [
        {'m': 'SiO2', 'd': 100}, 
        {'m': 'TiO2', 'd': 200}
    ], {'s': 1})
model_1.R(np.linspace(100,200))

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.08148791e-33, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.08148791e-33, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])